In [1]:
import requests as rq
from bs4 import BeautifulSoup
import json

In [2]:
#fetching the website
def get_link(link):
    web = rq.get(link)
    #print(web)
    soup = BeautifulSoup(web.content , "html.parser")
    #print(soup)
    return soup


In [3]:
#List's to store all the information
batting_summary=[]
bowling_summary=[]

In [4]:
#batting summary
def batting(batting_list , Team1 , Team2 , Teams):
    
    desc=["Batsman" , "Dismissal" , "Runs" , "Balls" ,"Minutes", "4s" , "6s" , "SR"] #attributes
    data=[]

    for number,k in enumerate(batting_list):

            table = k.find_all("tr")
            counter = 1 #to maintain batting position
            for i in table:
                row_data={}

                if(i.find("td").get_text().replace('\xa0', ' ').strip() == ''):  #to remove unnecessary blank spaces
                    continue

                row_data.update({"Match" : f"{Team1} vs {Team2}"})
                row_data.update({"Team-Innings" : f"{Teams[number]}"})  #to capture that Team's innings
                row_data.update({"Batting-Pos" : f"{counter}"})
                counter+=1

                for j,cell in enumerate(i.find_all("td")):
                    if(cell.get_text().replace('\xa0', ' ').strip() == ''):  #to remove unnecessary blank spaces
                        continue
                    row_data[desc[j]] = cell.get_text().replace('\xa0', ' ').strip()


                data.append(row_data)
            data.pop()#to get rid of uneccesary table row data
            data.pop()
            data.pop()

    data.pop()#to get rid of unnecessary table row data
    batting_summary.append(data)
        

In [5]:
#bowling summary
def bowling(bowling_list , Team1 , Team2 , Teams):
    
    desc = ["Bowler-name","overs","maiden","runs","wickets","economy","0s","4s","6s","wides" ,"no-balls"] #attributes
    data=[]
    Teams.reverse() #Team that bats second bowls first
    flag=0 #to remove unnecessary data
    for number,k in enumerate(bowling_list):

            table = k.find_all("tr")
            for i  in table:
                row_data={}

                if(flag==1):
                    flag=0
                    continue

                if(i.find("td").get_text().replace('\xa0', ' ').strip() == ''):
                    continue

                row_data.update({"Match" : f"{Team1} vs {Team2}"})
                row_data.update({"Bowling-Team" : f"{Teams[number]}"})

                for j,cell in enumerate(i.find_all("td")):
                    if(cell.get_text().replace('\xa0', ' ').strip() == ''):  #to remove unnecessary blank spaces
                        continue
                    if (j== 4 and int(cell.get_text().replace('\xa0', ' ').strip())>0):
                        flag=1  # #to remove unnecessary data
                    row_data[desc[j]] = cell.get_text().replace('\xa0', ' ').strip()


                data.append(row_data)
        
    bowling_summary.append(data)


In [6]:
def scrape(link):
    
    soup=get_link(link)
    body = soup.find_all('tbody')
    array = []

    for i in body:
        array.append(i)
    
    batting_list=[array[0] , array[2]] #contains batting summary of both innings
    bowling_list=[array[1] , array[3]] #contains bowling summary of both innings

    Teams=[] 
    for i,text in enumerate(soup.find_all('span' , class_ = "ds-text-title-xs ds-font-bold ds-capitalize")):
        Teams.append(text.get_text())

    Team1=Teams[0]
    Team2=Teams[1]
    
    batting(batting_list , Team1 , Team2 , Teams)
    bowling(bowling_list , Team1 , Team2 , Teams)
    

In [7]:
#navigator code
soup = get_link("https://www.espncricinfo.com/records/tournament/team-match-results/icc-cricket-world-cup-2023-24-15338")
table = soup.table
rows = table.find_all("tr")
total_matches = len(rows) - 1  #total amount of matches
#print(total_matches) 

#For obtaining links of all matches

links = [] # array to store all the links
final=  {} #dictionary to store all the Match - links pair

part = "https://www.espncricinfo.com" #part of URL to be appended to each link as it is missing 
for row in range(len(rows)):
    if row == 0: 
        continue #first row of table is about the column description
    else:
        cols = rows[row].find_all('td')
        temp = part+cols[6].a['href']
        links.append(temp)


links.reverse() #to store links in ascending order starting from the first match to the final
for i,link in enumerate(links):
    
    final.update({f"Match{i +1}" : f"{link}"})  

In [8]:
#Capturing the summary of every match

data = []
columns = []
columns_row = table.find('tr')
for cell in columns_row.find_all('td'):
    columns.append(cell.get_text())

for row in rows[1:]:  #skip the column descriptors
    row_data = {}
    cells = row.find_all(b'td')
    for i, column in enumerate(columns):
        row_data[column] = cells[i].get_text()
    data.append(row_data)

# Output as JSON
with open('match_summaries.json', 'w') as f:
    json.dump(data, f, indent=4)

In [9]:
for i in final.values():
    scrape(i)

In [10]:
#dumping the final summary lists to a JSON file
with open(r"C:\Users\Ram\Desktop\Code\Projects\Cricket Data Analysis\bowling.json", "w") as json_file:
        json.dump(bowling_summary, json_file, indent=4)
with open(r"C:\Users\Ram\Desktop\Code\Projects\Cricket Data Analysis\batting.json", "w") as json_file:
        json.dump(batting_summary, json_file, indent=4)